## Run and Validate ELT Data Pipeline using Dataproc

Let us go ahead and run the ELT Data Pipeline using Dataproce Workflow Template and also validate to see if the Pipeline is successfully run as per the requirements. We will clean up all the workflow runs before taking care of the run and validation of the ELT Data Pipeline.
* Step 1: Pre-run Validation
* Step 2: Run ELT Data Pipeline using Dataproc Workflow Template
* Step 3: Post-run Validation

In [1]:
!gcloud dataproc 

ERROR: (gcloud.dataproc) Command name argument expected.

Available groups for gcloud dataproc:

      autoscaling-policies    Create and manage Dataproc autoscaling policies.
      batches                 Submit Dataproc batch jobs.
      clusters                Create and manage Dataproc clusters.
      jobs                    Submit and manage Dataproc jobs.
      node-groups             Manage Dataproc node groups.
      operations              View and manage Dataproc operations.
      workflow-templates      Create and manage Dataproc workflow templates.

For detailed information on this command and its flags, run:
  gcloud dataproc --help


In [2]:
!gcloud dataproc operations

ERROR: (gcloud.dataproc.operations) Command name argument expected.

Available commands for gcloud dataproc operations:

      cancel                  Cancel an active operation.
      delete                  Delete the record of an inactive operation.
      describe                View the details of an operation.
      get-iam-policy          Get IAM policy for an operation.
      list                    View the list of all operations.
      set-iam-policy          Set IAM policy for an operation.

For detailed information on this command and its flags, run:
  gcloud dataproc operations --help


In [14]:
!gcloud dataproc operations list

Listed 0 items.


In [4]:
!gcloud dataproc operations describe 8d3e85c4-768e-34cb-8e6f-3bdf7f916916

done: true
metadata:
  '@type': type.googleapis.com/google.cloud.dataproc.v1.WorkflowMetadata
  clusterUuid: d1f87e3a-5d73-4d88-b893-fc5dee563e03
  endTime: '2024-04-21T07:30:35.777976Z'
  graph:
    nodes:
    - jobId: job-cleanup-ofco6qajm5lzy
      state: COMPLETED
      stepId: job-cleanup
    - jobId: job-convert-orders-ofco6qajm5lzy
      prerequisiteStepIds:
      - job-cleanup
      state: COMPLETED
      stepId: job-convert-orders
    - jobId: job-convert-order-items-ofco6qajm5lzy
      prerequisiteStepIds:
      - job-cleanup
      state: COMPLETED
      stepId: job-convert-order-items
    - jobId: job-daily-product-revenue-ofco6qajm5lzy
      prerequisiteStepIds:
      - job-convert-orders
      - job-convert-order-items
      state: COMPLETED
      stepId: job-daily-product-revenue
  startTime: '2024-04-21T07:27:57.378621Z'
  state: DONE
  template: wf-daily-product-revenue
  version: 6
name: projects/wired-method-417107/regions/asia-southeast2/operations/8d3e85c4-768e-34cb

In [13]:
!gcloud dataproc operations delete f264d113-9047-472e-9109-1062dd6859d2 --quiet

Deleted [f264d113-9047-472e-9109-1062dd6859d2].


In [15]:
!gcloud dataproc jobs submit spark-sql --cluster=khazdataprocdev -f gs://khazretail/scripts/daily_product_revenue/cleanup.sql

done: true
driverControlFilesUri: gs://dataproc-staging-asia-southeast2-138920929384-yzz5grop/google-cloud-dataproc-metainfo/d1f87e3a-5d73-4d88-b893-fc5dee563e03/jobs/4f5312af94d64609b6b2aa065e25dc42/
driverOutputResourceUri: gs://dataproc-staging-asia-southeast2-138920929384-yzz5grop/google-cloud-dataproc-metainfo/d1f87e3a-5d73-4d88-b893-fc5dee563e03/jobs/4f5312af94d64609b6b2aa065e25dc42/driveroutput
jobUuid: e405f007-fd9c-35a3-8aa2-36a24c9cb031
placement:
  clusterName: khazdataprocdev
  clusterUuid: d1f87e3a-5d73-4d88-b893-fc5dee563e03
reference:
  jobId: 4f5312af94d64609b6b2aa065e25dc42
  projectId: wired-method-417107
sparkSqlJob:
  queryFileUri: gs://khazretail/scripts/daily_product_revenue/cleanup.sql
status:
  state: DONE
  stateStartTime: '2024-04-21T07:43:41.790480Z'
statusHistory:
- state: PENDING
  stateStartTime: '2024-04-21T07:43:14.140709Z'
- state: SETUP_DONE
  stateStartTime: '2024-04-21T07:43:14.178023Z'
- details: Agent reported job success
  state: RUNNING
  stateSt

Job [4f5312af94d64609b6b2aa065e25dc42] submitted.
Waiting for job output...
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
ivysettings.xml file not found in HIVE_HOME or HIVE_CONF_DIR,/etc/hive/conf.dist/ivysettings.xml will be used
24/04/21 07:43:22 INFO SparkEnv: Registering MapOutputTracker
24/04/21 07:43:22 INFO SparkEnv: Registering BlockManagerMaster
24/04/21 07:43:22 INFO SparkEnv: Registering BlockManagerMasterHeartbeat
24/04/21 07:43:22 INFO SparkEnv: Registering OutputCommitCoordinator
Spark Web UI available at http://khazdataprocdev-m.asia-southeast2-c.c.wired-method-417107.internal:33641
Spark master: yarn, Application Id: application_1711307687799_0047
Time taken: 6.464 seconds
Time taken: 0.52 seconds
Job [4f5312af94d64609b6b2aa065e25dc42] finished successfully.


In [16]:
!gcloud dataproc workflow-templates describe wf-daily-product-revenue

createTime: '2024-04-21T07:04:04.406256Z'
id: wf-daily-product-revenue
jobs:
- sparkSqlJob:
    queryFileUri: gs://khazretail/scripts/daily_product_revenue/cleanup.sql
  stepId: job-cleanup
- prerequisiteStepIds:
  - job-cleanup
  sparkSqlJob:
    queryFileUri: gs://khazretail/scripts/daily_product_revenue/file_format_converter.sql
    scriptVariables:
      bucket_name: gs://khazretail
      table_name: orders
  stepId: job-convert-orders
- prerequisiteStepIds:
  - job-cleanup
  sparkSqlJob:
    queryFileUri: gs://khazretail/scripts/daily_product_revenue/file_format_converter.sql
    scriptVariables:
      bucket_name: gs://khazretail
      table_name: order_items
  stepId: job-convert-order-items
- prerequisiteStepIds:
  - job-convert-orders
  - job-convert-order-items
  sparkSqlJob:
    queryFileUri: gs://khazretail/scripts/daily_product_revenue/compute_daily_product_revenue.sql
    scriptVariables:
      bucket_name: gs://khazretail
  stepId: job-daily-product-revenue
name: project

In [17]:
!gcloud dataproc workflow-templates instantiate wf-daily-product-revenue

Waiting on operation [projects/wired-method-417107/regions/asia-southeast2/operations/a75cbe2a-84c2-3ec2-a46b-a45242d31fd9].
WorkflowTemplate [wf-daily-product-revenue] RUNNING
Job ID job-cleanup-ghg6ir5m2kcgy RUNNING
Job ID job-cleanup-ghg6ir5m2kcgy COMPLETED
Job ID job-convert-orders-ghg6ir5m2kcgy RUNNING
Job ID job-convert-order-items-ghg6ir5m2kcgy RUNNING
Job ID job-convert-order-items-ghg6ir5m2kcgy COMPLETED
Job ID job-convert-orders-ghg6ir5m2kcgy COMPLETED
Job ID job-daily-product-revenue-ghg6ir5m2kcgy RUNNING
WorkflowTemplate [wf-daily-product-revenue] DONE
Job ID job-daily-product-revenue-ghg6ir5m2kcgy COMPLETED


In [19]:
!gcloud dataproc jobs submit spark-sql --cluster=khazdataprocdev -e "SHOW databases"

done: true
driverControlFilesUri: gs://dataproc-staging-asia-southeast2-138920929384-yzz5grop/google-cloud-dataproc-metainfo/d1f87e3a-5d73-4d88-b893-fc5dee563e03/jobs/f05ace9b6958472d89f198540965f921/
driverOutputResourceUri: gs://dataproc-staging-asia-southeast2-138920929384-yzz5grop/google-cloud-dataproc-metainfo/d1f87e3a-5d73-4d88-b893-fc5dee563e03/jobs/f05ace9b6958472d89f198540965f921/driveroutput
jobUuid: be4c60fd-67ab-3c73-bd38-2dd1420e93c8
placement:
  clusterName: khazdataprocdev
  clusterUuid: d1f87e3a-5d73-4d88-b893-fc5dee563e03
reference:
  jobId: f05ace9b6958472d89f198540965f921
  projectId: wired-method-417107
sparkSqlJob:
  queryList:
    queries:
    - SHOW databases
status:
  state: DONE
  stateStartTime: '2024-04-21T07:51:21.405237Z'
statusHistory:
- state: PENDING
  stateStartTime: '2024-04-21T07:50:56.051913Z'
- state: SETUP_DONE
  stateStartTime: '2024-04-21T07:50:56.085984Z'
- details: Agent reported job success
  state: RUNNING
  stateStartTime: '2024-04-21T07:50:

Job [f05ace9b6958472d89f198540965f921] submitted.
Waiting for job output...
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
ivysettings.xml file not found in HIVE_HOME or HIVE_CONF_DIR,/etc/hive/conf.dist/ivysettings.xml will be used
24/04/21 07:51:03 INFO SparkEnv: Registering MapOutputTracker
24/04/21 07:51:03 INFO SparkEnv: Registering BlockManagerMaster
24/04/21 07:51:03 INFO SparkEnv: Registering BlockManagerMasterHeartbeat
24/04/21 07:51:03 INFO SparkEnv: Registering OutputCommitCoordinator
Spark Web UI available at http://khazdataprocdev-m.asia-southeast2-c.c.wired-method-417107.internal:37081
Spark master: yarn, Application Id: application_1711307687799_0052
default
retail_bronze_db
retail_gold_db
Time taken: 6.129 seconds, Fetched 3 row(s)
Job [f05ace9b6958472d89f198540965f921] finished successfully.


In [20]:
!gcloud dataproc jobs submit spark-sql --cluster=khazdataprocdev -e "USE retail_bronze_db;SHOW tables"

done: true
driverControlFilesUri: gs://dataproc-staging-asia-southeast2-138920929384-yzz5grop/google-cloud-dataproc-metainfo/d1f87e3a-5d73-4d88-b893-fc5dee563e03/jobs/3b57877c821e40c183427f9a57b1225c/
driverOutputResourceUri: gs://dataproc-staging-asia-southeast2-138920929384-yzz5grop/google-cloud-dataproc-metainfo/d1f87e3a-5d73-4d88-b893-fc5dee563e03/jobs/3b57877c821e40c183427f9a57b1225c/driveroutput
jobUuid: 523a1b41-a4d4-317f-9a59-35e755e49060
placement:
  clusterName: khazdataprocdev
  clusterUuid: d1f87e3a-5d73-4d88-b893-fc5dee563e03
reference:
  jobId: 3b57877c821e40c183427f9a57b1225c
  projectId: wired-method-417107
sparkSqlJob:
  queryList:
    queries:
    - USE retail_bronze_db;SHOW tables
status:
  state: DONE
  stateStartTime: '2024-04-21T07:52:56.441819Z'
statusHistory:
- state: PENDING
  stateStartTime: '2024-04-21T07:52:26.483189Z'
- state: SETUP_DONE
  stateStartTime: '2024-04-21T07:52:26.516224Z'
- details: Agent reported job success
  state: RUNNING
  stateStartTime: 

Job [3b57877c821e40c183427f9a57b1225c] submitted.
Waiting for job output...
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
ivysettings.xml file not found in HIVE_HOME or HIVE_CONF_DIR,/etc/hive/conf.dist/ivysettings.xml will be used
24/04/21 07:52:33 INFO SparkEnv: Registering MapOutputTracker
24/04/21 07:52:34 INFO SparkEnv: Registering BlockManagerMaster
24/04/21 07:52:34 INFO SparkEnv: Registering BlockManagerMasterHeartbeat
24/04/21 07:52:34 INFO SparkEnv: Registering OutputCommitCoordinator
Spark Web UI available at http://khazdataprocdev-m.asia-southeast2-c.c.wired-method-417107.internal:46279
Spark master: yarn, Application Id: application_1711307687799_0053
Time taken: 4.578 seconds
order_items
orders
Time taken: 2.156 seconds, Fetched 2 row(s)
Job [3b57877c821e40c183427f9a57b1225c] finished successfully.


In [21]:
!gcloud dataproc jobs submit spark-sql --cluster=khazdataprocdev -e "USE retail_bronze_db;SELECT count(*) FROM orders"

done: true
driverControlFilesUri: gs://dataproc-staging-asia-southeast2-138920929384-yzz5grop/google-cloud-dataproc-metainfo/d1f87e3a-5d73-4d88-b893-fc5dee563e03/jobs/bd3c97171c03464da1a5dae027b3a533/
driverOutputResourceUri: gs://dataproc-staging-asia-southeast2-138920929384-yzz5grop/google-cloud-dataproc-metainfo/d1f87e3a-5d73-4d88-b893-fc5dee563e03/jobs/bd3c97171c03464da1a5dae027b3a533/driveroutput
jobUuid: 22c53fb5-3760-3104-ae06-f08307a6d8c8
placement:
  clusterName: khazdataprocdev
  clusterUuid: d1f87e3a-5d73-4d88-b893-fc5dee563e03
reference:
  jobId: bd3c97171c03464da1a5dae027b3a533
  projectId: wired-method-417107
sparkSqlJob:
  queryList:
    queries:
    - USE retail_bronze_db;SELECT count(*) FROM orders
status:
  state: DONE
  stateStartTime: '2024-04-21T07:54:01.496500Z'
statusHistory:
- state: PENDING
  stateStartTime: '2024-04-21T07:53:23.431421Z'
- state: SETUP_DONE
  stateStartTime: '2024-04-21T07:53:23.463862Z'
- details: Agent reported job success
  state: RUNNING
  

Job [bd3c97171c03464da1a5dae027b3a533] submitted.
Waiting for job output...
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
ivysettings.xml file not found in HIVE_HOME or HIVE_CONF_DIR,/etc/hive/conf.dist/ivysettings.xml will be used
24/04/21 07:53:31 INFO SparkEnv: Registering MapOutputTracker
24/04/21 07:53:31 INFO SparkEnv: Registering BlockManagerMaster
24/04/21 07:53:31 INFO SparkEnv: Registering BlockManagerMasterHeartbeat
24/04/21 07:53:31 INFO SparkEnv: Registering OutputCommitCoordinator
Spark Web UI available at http://khazdataprocdev-m.asia-southeast2-c.c.wired-method-417107.internal:38477
Spark master: yarn, Application Id: application_1711307687799_0054
Time taken: 5.073 seconds
68883
Time taken: 10.807 seconds, Fetched 1 row(s)
Job [bd3c97171c03464da1a5dae027b3a533] finished successfully.


In [22]:
!gcloud dataproc jobs submit spark-sql --cluster=khazdataprocdev -e "USE retail_gold_db;SHOW tables"

done: true
driverControlFilesUri: gs://dataproc-staging-asia-southeast2-138920929384-yzz5grop/google-cloud-dataproc-metainfo/d1f87e3a-5d73-4d88-b893-fc5dee563e03/jobs/dd6d88e86c204db4868801538877b7be/
driverOutputResourceUri: gs://dataproc-staging-asia-southeast2-138920929384-yzz5grop/google-cloud-dataproc-metainfo/d1f87e3a-5d73-4d88-b893-fc5dee563e03/jobs/dd6d88e86c204db4868801538877b7be/driveroutput
jobUuid: 6dd8d56a-28a8-3c09-9f3b-9ad35bbe6b7f
placement:
  clusterName: khazdataprocdev
  clusterUuid: d1f87e3a-5d73-4d88-b893-fc5dee563e03
reference:
  jobId: dd6d88e86c204db4868801538877b7be
  projectId: wired-method-417107
sparkSqlJob:
  queryList:
    queries:
    - USE retail_gold_db;SHOW tables
status:
  state: DONE
  stateStartTime: '2024-04-21T07:54:56.521041Z'
statusHistory:
- state: PENDING
  stateStartTime: '2024-04-21T07:54:28.334290Z'
- state: SETUP_DONE
  stateStartTime: '2024-04-21T07:54:28.369184Z'
- details: Agent reported job success
  state: RUNNING
  stateStartTime: '2

Job [dd6d88e86c204db4868801538877b7be] submitted.
Waiting for job output...
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
ivysettings.xml file not found in HIVE_HOME or HIVE_CONF_DIR,/etc/hive/conf.dist/ivysettings.xml will be used
24/04/21 07:54:35 INFO SparkEnv: Registering MapOutputTracker
24/04/21 07:54:36 INFO SparkEnv: Registering BlockManagerMaster
24/04/21 07:54:36 INFO SparkEnv: Registering BlockManagerMasterHeartbeat
24/04/21 07:54:36 INFO SparkEnv: Registering OutputCommitCoordinator
Spark Web UI available at http://khazdataprocdev-m.asia-southeast2-c.c.wired-method-417107.internal:41331
Spark master: yarn, Application Id: application_1711307687799_0055
Time taken: 5.479 seconds
daily_product_revenue
Time taken: 3.188 seconds, Fetched 1 row(s)
Job [dd6d88e86c204db4868801538877b7be] finished successfully.


In [23]:
!gcloud dataproc jobs submit spark-sql --cluster=khazdataprocdev -e "USE retail_gold_db;SELECT * FROM daily_product_revenue LIMIT 10"

done: true

Job [77bafe026b2c4fd3993f313a75974445] submitted.
Waiting for job output...
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
ivysettings.xml file not found in HIVE_HOME or HIVE_CONF_DIR,/etc/hive/conf.dist/ivysettings.xml will be used
24/04/21 07:55:28 INFO SparkEnv: Registering MapOutputTracker
24/04/21 07:55:29 INFO SparkEnv: Registering BlockManagerMaster
24/04/21 07:55:29 INFO SparkEnv: Registering BlockManagerMasterHeartbeat
24/04/21 07:55:29 INFO SparkEnv: Registering OutputCommitCoordinator
Spark Web UI available at http://khazdataprocdev-m.asia-southeast2-c.c.wired-method-417107.internal:43759
Spark master: yarn, Application Id: application_1711307687799_0056
Time taken: 5.59 seconds
2013-08-09 00:00:00.0	1073	PENDING	3	599.97
2013-08-11 00:00:00.0	365	COMPLETE	33	1979.67
2013-08-14 00:00:00.0	403	CANCELED	1	129.99
2013-09-04 00:00:00.0	703	PENDING_PAYMENT	4	79.96
2013-10-07 00:00:00.0	191	PENDING_


driverControlFilesUri: gs://dataproc-staging-asia-southeast2-138920929384-yzz5grop/google-cloud-dataproc-metainfo/d1f87e3a-5d73-4d88-b893-fc5dee563e03/jobs/77bafe026b2c4fd3993f313a75974445/
driverOutputResourceUri: gs://dataproc-staging-asia-southeast2-138920929384-yzz5grop/google-cloud-dataproc-metainfo/d1f87e3a-5d73-4d88-b893-fc5dee563e03/jobs/77bafe026b2c4fd3993f313a75974445/driveroutput
jobUuid: 28ed1467-d39d-3fa6-b8cb-8325f087c460
placement:
  clusterName: khazdataprocdev
  clusterUuid: d1f87e3a-5d73-4d88-b893-fc5dee563e03
reference:
  jobId: 77bafe026b2c4fd3993f313a75974445
  projectId: wired-method-417107
sparkSqlJob:
  queryList:
    queries:
    - USE retail_gold_db;SELECT * FROM daily_product_revenue LIMIT 10
status:
  state: DONE
  stateStartTime: '2024-04-21T07:55:56.578256Z'
statusHistory:
- state: PENDING
  stateStartTime: '2024-04-21T07:55:21.604373Z'
- state: SETUP_DONE
  stateStartTime: '2024-04-21T07:55:21.634005Z'
- details: Agent reported job success
  state: RUNNI